In this qucikstart we'll see how to :
- get setup with LangChain, LangSmith and LangServe
- USe the most basic and common components of Langchain: Prompt templates , models and output parser.
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o")
print(llm)

In [ ]:
## Input and get resposne from LLM
result = llm.invoke("What is generative AI?")
result

In [ ]:
### Chatpromt Template
from langchain_core.promts import ChatPromtTemplate
prompt = ChatPromtTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer. Provide me answer based on the question"),
        ("user","{input}")
    ]
)
prompt

In [ ]:
## Chain
chain = prompt|llm
response = chain.invoke({"input":"can you tell me about Langsmith?"})
response

In [ ]:
## type = AIMessage
type(response)

In [ ]:
## stroutput Parser
## convert the output content to string
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | llm | output_parser  # first go to prompt then llm then output_parser

response = chain.invoke({"input":"Can you tell me about Langsmith?"})
response